In [7]:
# Imports here
import torch
import PIL
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
from collections import OrderedDict
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/f7/27/c0faba9135bf3f110810e7e7896233c92edb92827ef824649f08d24adebd/torchvision-0.16.2-cp310-cp310-win_amd64.whl.metadata
  Using cached torchvision-0.16.2-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
Using cached torchvision-0.16.2-cp310-cp310-win_amd64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\monis\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [8]:

data_dir = 'C:/Monish/New folder/Prj/AIML Lab 7th sem/fruit images'
train_dir = data_dir + '/train/train'

# Define your transformations
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load all the images from the train folder
all_data = datasets.ImageFolder(train_dir, transform=train_transforms)

# Calculate the sizes for train, validation, and test sets
total_size = len(all_data)
train_size = int(0.7 * total_size)
test_size = int(0.2 * total_size)
valid_size = total_size - train_size - test_size

# Use random_split to split the dataset
train_data, valid_data, test_data = torch.utils.data.random_split(all_data, [train_size, valid_size, test_size])

# Create data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=False)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=50)
testloader = torch.utils.data.DataLoader(test_data, batch_size=50)


In [ ]:
# image,data = train_data[2]
# image = np.transpose(image.numpy(), (1, 2, 0))

# plt.imshow(image)
# plt.show()

In [ ]:
# data

In [ ]:
# # Note that length of possible outpouts is 102
# import json
# with open('cat_to_name.json', 'r') as f:
#     cat_to_name = json.load(f)

**Opting for Keras model of the 16-layer network (vgg16) architecture over 13-layer network (vgg13).** [Reference](https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3)

In [9]:
# Load a pre-trained network 
model = models.alexnet(pretrained=True)
model.name = "alexnet"
model

C:\Users\monis\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\monis\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\monis/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|███████████████████████████████████████████████████████████████████████████████| 233M/233M [00:41<00:00, 5.86MB/s]


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

In [13]:
# Define a new, untrainted feed-forward network as a classifier, using ReLU activations and dropout
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(9216, 4096, bias=True)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.5)),
                          ('fc2', nn.Linear(4096, 33, bias=True)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [14]:
# Device agnostic code, automatically uses CUDA if it's enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [16]:
# change to device
model.to(device);

In [17]:
# Define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Define deep learning method
epochs = 5
print_every = 30 # Prints every 30 images out of batch of 50 images
steps = 0

In [18]:
# Implement a function for the validation pass
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels) in enumerate(testloader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [20]:
# Train the classifier layers using backpropogation using the pre-trained network to get features

print("Training process initializing .....\n")

for e in range(epochs):
    running_loss = 0
    model.train() # Technically not necessary, setting this for good measure
    
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()

            with torch.no_grad():
                valid_loss, accuracy = validation(model, validloader, criterion)
            
            print("Epoch: {}/{} | ".format(e+1, epochs),
                  "Training Loss: {:.4f} | ".format(running_loss/print_every),
                  "Validation Loss: {:.4f} | ".format(valid_loss/len(validloader)),
                  "Validation Accuracy: {:.4f}".format(accuracy/len(validloader)))
            
            running_loss = 0
            model.train()

print("\nTraining process is now complete!!")

Training process initializing .....

Epoch: 1/5 |  Training Loss: 0.4663 |  Validation Loss: 0.2812 |  Validation Accuracy: 0.9192
Epoch: 1/5 |  Training Loss: 0.3258 |  Validation Loss: 0.2252 |  Validation Accuracy: 0.9290
Epoch: 1/5 |  Training Loss: 0.2598 |  Validation Loss: 0.2110 |  Validation Accuracy: 0.9357
Epoch: 1/5 |  Training Loss: 0.2333 |  Validation Loss: 0.2214 |  Validation Accuracy: 0.9282
Epoch: 1/5 |  Training Loss: 0.2394 |  Validation Loss: 0.1388 |  Validation Accuracy: 0.9565
Epoch: 1/5 |  Training Loss: 0.2780 |  Validation Loss: 0.2048 |  Validation Accuracy: 0.9292
Epoch: 1/5 |  Training Loss: 0.2693 |  Validation Loss: 0.2057 |  Validation Accuracy: 0.9394
Epoch: 2/5 |  Training Loss: 0.0355 |  Validation Loss: 0.1682 |  Validation Accuracy: 0.9484
Epoch: 2/5 |  Training Loss: 0.2767 |  Validation Loss: 0.1312 |  Validation Accuracy: 0.9608
Epoch: 2/5 |  Training Loss: 0.3010 |  Validation Loss: 0.1899 |  Validation Accuracy: 0.9512
Epoch: 2/5 |  Training 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [21]:
all_labels = []
all_predictions = []

Try to load all image tensors to gpu before the loop

In [22]:
# TODO: Do validation on the test set
# Do validation on the test set
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data
        all_labels.extend(labels.numpy())
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_tensor_cpu = predicted.to('cpu')
        all_predictions.extend(predicted_tensor_cpu.numpy())

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct / total))

Time:  87.8890392780304
Accuracy achieved by the network on test images is: 97%


In [23]:
print(len(all_labels))

3370


In [24]:
print(len(all_predictions))

3370


In [25]:
from sklearn.metrics import confusion_matrix
import numpy as np
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate the confusion matrix
cm = confusion_matrix(all_labels, all_predictions)

print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[ 91   0   0 ...   0   0   0]
 [  0 104   0 ...   0   0   0]
 [  0   0 104 ...   0   2   0]
 ...
 [  0   0   0 ... 107   0   0]
 [  0   0   0 ...   0 154   0]
 [  0   0   0 ...   0   0  72]]


In [26]:
from sklearn.metrics import classification_report
report = classification_report(all_labels, all_predictions)
print(report)

              precision    recall  f1-score   support

           0       0.84      0.97      0.90        94
           1       0.91      1.00      0.95       104
           2       0.97      0.98      0.98       106
           3       0.99      1.00      0.99        94
           4       0.99      0.94      0.96        96
           5       1.00      0.98      0.99       110
           6       0.99      0.99      0.99        95
           7       0.99      1.00      0.99        89
           8       0.96      1.00      0.98        89
           9       1.00      1.00      1.00       103
          10       1.00      0.96      0.98        81
          11       0.99      1.00      0.99        79
          12       0.99      0.99      0.99       199
          13       1.00      0.98      0.99        92
          14       0.97      1.00      0.99        99
          15       1.00      0.97      0.98        92
          16       1.00      0.95      0.98       105
          17       0.97    

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

To Create a checkpoint [Reference](https://discuss.pytorch.org/t/saving-and-loading-a-model-in-pytorch/2610)

In [27]:
# Assuming your train_data is a Subset object
original_dataset = train_data.dataset

# Create a class_to_idx mapping
model.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}


In [28]:
print(model.class_to_idx)

{'Apple Braeburn': 0, 'Apple Granny Smith': 1, 'Apricot': 2, 'Avocado': 3, 'Banana': 4, 'Blueberry': 5, 'Cactus fruit': 6, 'Cantaloupe': 7, 'Cherry': 8, 'Clementine': 9, 'Corn': 10, 'Cucumber Ripe': 11, 'Grape Blue': 12, 'Kiwi': 13, 'Lemon': 14, 'Limes': 15, 'Mango': 16, 'Onion White': 17, 'Orange': 18, 'Papaya': 19, 'Passion Fruit': 20, 'Peach': 21, 'Pear': 22, 'Pepper Green': 23, 'Pepper Red': 24, 'Pineapple': 25, 'Plum': 26, 'Pomegranate': 27, 'Potato Red': 28, 'Raspberry': 29, 'Strawberry': 30, 'Tomato': 31, 'Watermelon': 32}


In [29]:
checkpoint = {'architecture': model.name,
             'classifier': model.classifier,
             'class_to_idx': model.class_to_idx,
             'state_dict': model.state_dict()}

torch.save(checkpoint, 'model.pth')